## Modele de clasificare
Folositi 4 seturi de date pentru probleme de clasificare, plecand de la repository-urile specificate in Cursul 6. Cel putin un set de date sa fie cu valori lipsa; pentru un alt set de date care are initial toate valorile, introduceti dvs. in mod artificial valori lipsa, suprascriind un anumit procent din valorile initiale (ex. p=5%, p parametru) cu numpy.nan.

In [1189]:
import numpy as np
print(f'NumPy version: {np.__version__}')

import pandas as pd
print(f'Pandas version: {pd.__version__}')

from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import sklearn as sk
print(f'Sklearn version: {sk.__version__}')

#NumPy version: 1.20.1
#Pandas version: 1.2.3
#Sklearn version: 0.24.1

NumPy version: 1.20.1
Pandas version: 1.2.3
Sklearn version: 0.24.1


In [1190]:
le=LabelEncoder()

In [1191]:
iris: pd.core.frame.DataFrame = pd.read_csv("Data/iris.data", header=None)
x_iris: np.array = iris.iloc[:, :-1].values
y_iris: np.array = iris[[4]].apply(le.fit_transform).values
y_iris = y_iris.reshape(y_iris.shape[0])

In [1192]:
wine:pd.core.frame.DataFrame = pd.read_csv("Data/wine.data", header=None)
x_wine: np.array = wine.iloc[:, 1:].values
y_wine: np.array = wine.iloc[:, 0].values

In [1193]:
wdbc: pd.core.frame.DataFrame = pd.read_csv("Data/wdbc.data", header=None)  
x_wdbc: np.array = wdbc.iloc[:, 2:].values
y_wdbc: np.array = wdbc[[1]].apply(le.fit_transform).values
y_wdbc = y_wdbc.reshape(y_wdbc.shape[0])

In [1194]:
abalone: pd.core.frame.DataFrame = pd.read_csv("Data/abalone.data", header=None) 
x_abalone: np.array = abalone.iloc[:, 1:].values
y_abalone: np.array = abalone[[0]].apply(le.fit_transform).values
y_abalone = y_abalone.reshape(y_abalone.shape[0])

1. (20 puncte) Aplicati o metoda de missing value imputation, unde este cazul; documentati metoda folosita.
*Resurse*: Pentru missing value imputation, puteti urmari [Imputation of missing values](https://scikit-learn.org/stable/modules/impute.html), [How to Handle Missing Data with Python](https://machinelearningmastery.com/handle-missing-data-python/), [fancyimpute](https://github.com/iskandr/fancyimpute), [missingpy](https://github.com/epsilon-machine/missingpy).

*Cerinta*: In cazul in care folositi un pachet ce trebuie instalat (nu face parte din distributia standard anaconda), includeti intr-o celula o comanda de instalare corespunzatoare folosind semn de exclamare, de exemplu:
```python
!pip install missingpy
```
(sursa: [https://github.com/epsilon-machine/missingpy](https://github.com/epsilon-machine/missingpy)). La executia celulei in Jupyter Notebook se instaleaza pachetul, iar in celulele ulterioare importurile din noul pachet functioneaza. 

In [1195]:
def random_nans(data_set: np.array, percentage: int = 5) -> np.array:
    """
    Adding NaN values in a dataset.
    
    Args:
        data_set:the numpy.ndarray containing the dataset
        percentage:the percentage(from dataset size) of Nans values to be added in the dataset
        
    Returns:
        the modified dataset
    """
    result_set: np.array = data_set.copy()
    choice: np.array = np.random.choice(result_set.size, int(percentage / 100 * result_set.size), replace=False)
    rows, cols = np.unravel_index(choice, result_set.shape)   
    result_set[rows, cols] = np.nan
    return result_set

In [1196]:
x_iris = random_nans(x_iris)
x_wine = random_nans(x_wine)
x_wdbc = random_nans(x_wdbc)
x_abalone = random_nans(x_abalone)

In [1197]:
imp = SimpleImputer(missing_values = np.nan, strategy="most_frequent")
x_iris = imp.fit_transform(x_iris)
x_wine = imp.fit_transform(x_wine)
x_wdbc = imp.fit_transform(x_wdbc)
x_abalone = imp.fit_transform(x_abalone)

2. (numar de modele * numar de seturi de date \* 1 punct = 20 de puncte) Pentru fiecare set de date aplicati 5 modele de clasificare din scikit learn. Pentru fiecare raportati: acuratete, precision, recall, scorul F1 - a se vedea [sklearn.metrics](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics), [Precision and recall](https://en.wikipedia.org/wiki/Precision_and_recall) - folosind 5 fold cross validation. Raportati mediile rezultatelor atat pentru fold-urile de antrenare, cat si pentru cele de testare. Rularile se vor face cu valori fixate ale hiperparametrilor. 

In [1198]:
def show_metrics_classifier(clf,x,y):
    scoring=['recall_macro','precision_macro','accuracy','f1_macro']
    scores = cross_validate(clf,x, y, cv=5,scoring=scoring,return_train_score=True)
    df = pd.DataFrame(data={'train_accuracy': scores['train_accuracy'],
                            'train_recall': scores['train_recall_macro'],
                            'train_precision': scores['train_precision_macro'],
                            'train_f1': scores['train_f1_macro'],
                            'test_accuracy': scores['test_accuracy'],
                            'test_recall': scores['test_recall_macro'],
                            'test_precision': scores['test_precision_macro'],
                            'test_f1': scores['test_f1_macro']
                           })
    df.loc['mean'] = df.mean()
    with pd.option_context('expand_frame_repr', False):
        print (df)

In [1199]:
print("Metrics for KNeighborsClassifier:")
print("\nIris_Dataset:")
show_metrics_classifier(KNeighborsClassifier(n_neighbors=5),x_iris,y_iris) 
print("\nWine_Dataset:")
show_metrics_classifier(KNeighborsClassifier(n_neighbors=5),x_wine ,y_wine )
print("\nWdbc_Dataset:")
show_metrics_classifier(KNeighborsClassifier(n_neighbors=5),x_wdbc ,y_wdbc )
print("\nAbalone_Dataset:")
show_metrics_classifier(KNeighborsClassifier(n_neighbors=5),x_abalone ,y_abalone )

Metrics for KNeighborsClassifier:

Iris_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0           0.950000      0.950000         0.950000  0.950000           0.90         0.90        0.899327  0.898162
1           0.958333      0.958333         0.958516  0.958327           0.90         0.90        0.923077  0.897698
2           0.958333      0.958333         0.958708  0.958215           0.90         0.90        0.923077  0.897698
3           0.966667      0.966667         0.966667  0.966667           0.90         0.90        0.902357  0.899749
4           0.941667      0.941667         0.941839  0.941658           1.00         1.00        1.000000  1.000000
mean        0.955000      0.955000         0.955146  0.954973           0.92         0.92        0.929567  0.918662

Wine_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0       

In [1200]:
print("Metrics for LogisticRegression:")
print("\nIris_Dataset:")
show_metrics_classifier(LogisticRegression(solver='sag', multi_class='multinomial', max_iter=10000),x_iris,y_iris) 
print("\nWine_Dataset:")
show_metrics_classifier(LogisticRegression(solver='sag', multi_class='multinomial', max_iter=10000),x_wine ,y_wine )
print("\nWdbc_Dataset:")
show_metrics_classifier(LogisticRegression(solver='sag', multi_class='multinomial', max_iter=10000),x_wdbc ,y_wdbc )
print("\nAbalone_Dataset:")
show_metrics_classifier(LogisticRegression(solver='sag', multi_class='multinomial', max_iter=10000),x_abalone ,y_abalone )

Metrics for LogisticRegression:

Iris_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0           0.916667      0.916667         0.916667  0.916667       0.800000     0.800000        0.795455  0.793651
1           0.891667      0.891667         0.891807  0.891650       0.866667     0.866667        0.904762  0.861111
2           0.883333      0.883333         0.883333  0.883333       0.866667     0.866667        0.875000  0.865320
3           0.925000      0.925000         0.926461  0.924894       0.866667     0.866667        0.866667  0.866667
4           0.875000      0.875000         0.875130  0.874980       0.966667     0.966667        0.969697  0.966583
mean        0.898333      0.898333         0.898680  0.898305       0.873333     0.873333        0.882316  0.870666

Wine_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0         

In [1201]:
print("Metrics for GaussianNB:")
print("\nIris_Dataset:")
show_metrics_classifier(GaussianNB(),x_iris,y_iris) 
print("\nWine_Dataset:")
show_metrics_classifier(GaussianNB(),x_wine ,y_wine )
print("\nWdbc_Dataset:")
show_metrics_classifier(GaussianNB(),x_wdbc ,y_wdbc )
print("\nAbalone_Dataset:")
show_metrics_classifier(GaussianNB(),x_abalone ,y_abalone )

Metrics for GaussianNB:

Iris_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0           0.933333      0.933333         0.933333  0.933333       0.800000     0.800000        0.825397  0.791667
1           0.908333      0.908333         0.912169  0.907974       0.933333     0.933333        0.933333  0.933333
2           0.925000      0.925000         0.926461  0.924894       0.900000     0.900000        0.923077  0.897698
3           0.933333      0.933333         0.934002  0.933292       0.866667     0.866667        0.866667  0.866667
4           0.900000      0.900000         0.900585  0.899937       0.966667     0.966667        0.969697  0.966583
mean        0.920000      0.920000         0.921310  0.919886       0.893333     0.893333        0.903634  0.891190

Wine_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0           0.9788

In [1202]:
print("Metrics for RandomForestClassifier:")
print("\n Iris_Dataset:")
show_metrics_classifier(RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1), x_iris, y_iris) 
print("\n Wine_Dataset:")
show_metrics_classifier(RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1), x_wine ,y_wine )
print("\n Wdbc_Dataset:")
show_metrics_classifier(RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1), x_wdbc ,y_wdbc )
print("\n Abalone_Dataset:")
show_metrics_classifier(RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1), x_abalone ,y_abalone )

Metrics for RandomForestClassifier:

 Iris_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0           0.975000      0.975000         0.975193  0.974996       0.900000     0.900000        0.907407  0.899522
1           0.991667      0.991667         0.991870  0.991665       0.966667     0.966667        0.969697  0.966583
2           1.000000      1.000000         1.000000  1.000000       0.933333     0.933333        0.944444  0.932660
3           0.983333      0.983333         0.984127  0.983323       0.933333     0.933333        0.933333  0.933333
4           0.983333      0.983333         0.984127  0.983323       1.000000     1.000000        1.000000  1.000000
mean        0.986667      0.986667         0.987063  0.986661       0.946667     0.946667        0.950976  0.946420

 Wine_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0   

In [1203]:
print("Metrics for DecisionTreeClassifier:")
print("\nIris_Dataset:")
show_metrics_classifier(DecisionTreeClassifier(max_depth=5),x_iris,y_iris) 
print("\nWine_Dataset:")
show_metrics_classifier(DecisionTreeClassifier(max_depth=5),x_wine ,y_wine )
print("\nWdbc_Dataset:")
show_metrics_classifier(DecisionTreeClassifier(max_depth=5),x_wdbc ,y_wdbc )
print("\nAbalone_Dataset:")
show_metrics_classifier(DecisionTreeClassifier(max_depth=5),x_abalone ,y_abalone )

Metrics for DecisionTreeClassifier:

Iris_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0           1.000000      1.000000         1.000000  1.000000       0.933333     0.933333        0.944444  0.932660
1           1.000000      1.000000         1.000000  1.000000       0.833333     0.833333        0.856061  0.832951
2           1.000000      1.000000         1.000000  1.000000       0.900000     0.900000        0.902357  0.899749
3           0.991667      0.991667         0.991870  0.991665       0.966667     0.966667        0.969697  0.966583
4           1.000000      1.000000         1.000000  1.000000       0.966667     0.966667        0.969697  0.966583
mean        0.998333      0.998333         0.998374  0.998333       0.920000     0.920000        0.928451  0.919705

Wine_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0     

3. (numar de modele * numar de seturi de date * 1 punct = 20 de puncte) Raportati performanta fiecarui model, folosind 5 fold cross validation. Pentru fiecare din cele 5 rulari, cautati hiperparametrii optimi folosind 4-fold cross validation. Performanta modelului va fi raportata ca medie a celor  5 rulari. 
    *Observatie:* la fiecare din cele 5 rulari, hiperparametrii optimi pot diferi, din cauza datelor utilizate pentru antrenare/validare. 

In [1204]:
def show_metrics_classifier_with_performance(clf, parameter_grid, x, y):
    show_metrics_classifier(GridSearchCV(estimator = clf, 
        param_grid=parameter_grid, cv=4, return_train_score=True), x, y)

In [189]:
print("Metrics for KNeighborsClassifier with performance:")
parameter_grid_KNeighborsClassifier = {'n_neighbors': list(range(1, 10)), 'p': [1, 2, 3, 10]}
print("\nIris_Dataset:")
show_metrics_classifier_with_performance(KNeighborsClassifier(), parameter_grid_KNeighborsClassifier, x_iris, y_iris) 
print("\nWine_Dataset:")
show_metrics_classifier_with_performance(KNeighborsClassifier(), parameter_grid_KNeighborsClassifier, x_wine, y_wine )
print("\nWdbc_Dataset:")
show_metrics_classifier_with_performance(KNeighborsClassifier(), parameter_grid_KNeighborsClassifier, x_wdbc, y_wdbc )
print("\nAbalone_Dataset:")
show_metrics_classifier_with_performance(KNeighborsClassifier(), parameter_grid_KNeighborsClassifier, x_abalone, y_abalone )

Metrics for KNeighborsClassifier with performance:

Iris_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0           0.916667      0.916667         0.920106  0.915948       0.933333     0.933333        0.933333  0.933333
1           0.941667      0.941667         0.943595  0.940678       0.966667     0.966667        0.969697  0.966583
2           1.000000      1.000000         1.000000  1.000000       0.900000     0.900000        0.902357  0.899749
3           1.000000      1.000000         1.000000  1.000000       0.833333     0.833333        0.835017  0.832916
4           0.941667      0.941667         0.941719  0.940991       1.000000     1.000000        1.000000  1.000000
mean        0.960000      0.960000         0.961084  0.959523       0.926667     0.926667        0.928081  0.926516

Wine_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision  

In [1229]:
print("Metrics for LogisticRegression with performance:")
parameter_grid_LogisticRegression = {
'solver':['newton-cg', 'sag', 'saga'],
'penalty' : ['l2'],
'C' :[100, 10, 1.0, 0.1, 0.01],'max_iter':[10000]}
print("\nIris_Dataset:")
show_metrics_classifier_with_performance(LogisticRegression(), parameter_grid_LogisticRegression, x_iris, y_iris) 
print("\nWine_Dataset:")
show_metrics_classifier_with_performance(LogisticRegression(), parameter_grid_LogisticRegression, x_wine, y_wine )
print("\nWdbc_Dataset:")
show_metrics_classifier_with_performance(LogisticRegression(), parameter_grid_LogisticRegression, x_wdbc, y_wdbc )
print("\nAbalone_Dataset:")
show_metrics_classifier_with_performance(LogisticRegression(), parameter_grid_LogisticRegression, x_abalone, y_abalone )

Metrics for LogisticRegression with performance:

Iris_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0           0.933333      0.933333         0.933333  0.933333       0.900000     0.900000        0.902357  0.899749
1           0.925000      0.925000         0.925162  0.924988       0.900000     0.900000        0.902357  0.899749
2           0.908333      0.908333         0.908484  0.908319       0.866667     0.866667        0.875000  0.865320
3           0.941667      0.941667         0.941839  0.941658       0.900000     0.900000        0.902357  0.899749
4           0.891667      0.891667         0.891807  0.891650       1.000000     1.000000        1.000000  1.000000
mean        0.920000      0.920000         0.920125  0.919990       0.913333     0.913333        0.916414  0.912914

Wine_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   t

In [1230]:
print("Metrics for GaussianNB with performance:")
parameter_grid_GaussianNB = {'var_smoothing': np.logspace(0,-9, num=100)}
print("\nIris_Dataset:")
show_metrics_classifier_with_performance(GaussianNB(), parameter_grid_GaussianNB, x_iris, y_iris) 
print("\nWine_Dataset:")
show_metrics_classifier_with_performance(GaussianNB(), parameter_grid_GaussianNB, x_wine, y_wine )
print("\nWdbc_Dataset:")
show_metrics_classifier_with_performance(GaussianNB(), parameter_grid_GaussianNB, x_wdbc, y_wdbc )
print("\nAbalone_Dataset:")
show_metrics_classifier_with_performance(GaussianNB(), parameter_grid_GaussianNB, x_abalone, y_abalone )

Metrics for GaussianNB with performance:

Iris_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0           0.941667      0.941667         0.941839  0.941658       0.800000     0.800000        0.825397  0.791667
1           0.925000      0.925000         0.925162  0.924988       0.933333     0.933333        0.944444  0.932660
2           0.933333      0.933333         0.933333  0.933333       0.900000     0.900000        0.923077  0.897698
3           0.933333      0.933333         0.934002  0.933292       0.900000     0.900000        0.902357  0.899749
4           0.900000      0.900000         0.900585  0.899937       0.966667     0.966667        0.969697  0.966583
mean        0.926667      0.926667         0.926984  0.926642       0.900000     0.900000        0.912994  0.897671

Wine_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0

In [188]:
print("Metrics for RandomForestClassifier with performance:")
parameter_grid_RandomForestClassifier = {'n_estimators':[10, 100, 1000],'max_features':['sqrt', 'log2']}
print("\nIris_Dataset:")
show_metrics_classifier_with_performance(RandomForestClassifier(), parameter_grid_RandomForestClassifier, x_iris, y_iris) 
print("\nWine_Dataset:")
show_metrics_classifier_with_performance(RandomForestClassifier(), parameter_grid_RandomForestClassifier, x_wine, y_wine )
print("\nWdbc_Dataset:")
show_metrics_classifier_with_performance(RandomForestClassifier(), parameter_grid_RandomForestClassifier, x_wdbc, y_wdbc )
print("\nAbalone_Dataset:")
show_metrics_classifier_with_performance(RandomForestClassifier(), parameter_grid_RandomForestClassifier, x_abalone, y_abalone )

Metrics for RandomForestClassifier with performance:

Iris_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0           1.000000      1.000000         1.000000  1.000000       0.966667     0.966667        0.969697  0.966583
1           1.000000      1.000000         1.000000  1.000000       0.966667     0.966667        0.969697  0.966583
2           1.000000      1.000000         1.000000  1.000000       0.933333     0.933333        0.944444  0.932660
3           0.991667      0.991667         0.991870  0.991665       0.900000     0.900000        0.899327  0.898162
4           1.000000      1.000000         1.000000  1.000000       1.000000     1.000000        1.000000  1.000000
mean        0.998333      0.998333         0.998374  0.998333       0.953333     0.953333        0.956633  0.952798

Wine_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision

In [1233]:
print("Metrics for DecisionTreeClassifier with performance:")
parameter_grid_KNeighborsClassifier = {'criterion':['gini', 'entropy'], 'max_depth':[2,4,6,8,10,12]}
print("\nIris_Dataset:")
show_metrics_classifier_with_performance(DecisionTreeClassifier(), parameter_grid_KNeighborsClassifier, x_iris, y_iris) 
print("\nWine_Dataset:")
show_metrics_classifier_with_performance(DecisionTreeClassifier(), parameter_grid_KNeighborsClassifier, x_wine, y_wine )
print("\nWdbc_Dataset:")
show_metrics_classifier_with_performance(DecisionTreeClassifier(), parameter_grid_KNeighborsClassifier, x_wdbc, y_wdbc )
print("\nAbalone_Dataset:")
show_metrics_classifier_with_performance(DecisionTreeClassifier(), parameter_grid_KNeighborsClassifier, x_abalone, y_abalone )

Metrics for DecisionTreeClassifier with performance:

Iris_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0               0.95          0.95         0.952862  0.949875       0.933333     0.933333        0.944444  0.932660
1               1.00          1.00         1.000000  1.000000       0.900000     0.900000        0.923077  0.897698
2               1.00          1.00         1.000000  1.000000       0.866667     0.866667        0.875000  0.865320
3               1.00          1.00         1.000000  1.000000       0.933333     0.933333        0.933333  0.933333
4               1.00          1.00         1.000000  1.000000       0.966667     0.966667        0.969697  0.966583
mean            0.99          0.99         0.990572  0.989975       0.920000     0.920000        0.929110  0.919119

Wine_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision